# $target

In [ ]:
%pylab inline

In [ ]:
from muscat2ta.transitanalysis import TransitAnalysis

In [ ]:
ta = TransitAnalysis(DATAROOT, '$target', '$night', TID, CIDS, use_opencl=True)

In [ ]:
fig = ta.phs[0].plot_raw(8, figsize=(13,5), ylim=(0.92, 1.05))

In [ ]:
ta.optimize('linear', niter = 3000)

In [ ]:
ta.plot_light_curve();

In [ ]:
ta.sample(repeats = 15)

In [ ]:
ta.plot_final_fit()

In [ ]:
ta.save()